# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [64]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import cv2
from PIL import Image
from skimage import io 
import skimage
import datetime
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

We set the random seed so that the results don't vary drastically.

In [65]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [66]:
train_doc = np.random.permutation(open('/kaggle/input/gesture-recognition-upgrad/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/kaggle/input/gesture-recognition-upgrad/Project_data/val.csv').readlines())
batch_size = 20#experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [67]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [1,3,4,5,7,8,9,10,12,14,15,16,18,20,22,24,26,28]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int((len(t) / batch_size))# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            print('batch',batch)
            batch_data = np.zeros((batch_size,18,120,120,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #print( 'image shape = ', image.shape)
                    image = cv2.resize(image,(120,120),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #print( 'image shape0 = ', image)
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.percentile(image[:,:,0],5))/ (np.percentile(image[:,:,0],95) - np.percentile(image[:,:,0],5))#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],5))/ (np.percentile(image[:,:,1],95) - np.percentile(image[:,:,1],5))#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],5))/ (np.percentile(image[:,:,2],95) - np.percentile(image[:,:,2],5)) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        if (len(t)%batch_size) != 0:
            print('last batch',len(t)%batch_size)
            print('num_batches',num_batches)
            batch_data = np.zeros((len(t)%batch_size,18,120,120,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                
                imgs = os.listdir(source_path+'/'+ t[folder + ((num_batches-1)*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    image = cv2.resize(image,(120,120),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)

                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.percentile(image[:,:,0],5))/ (np.percentile(image[:,:,0],95) - np.percentile(image[:,:,0],5))#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.percentile(image[:,:,1],5))/ (np.percentile(image[:,:,1],95) - np.percentile(image[:,:,1],5))#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.percentile(image[:,:,2],5))/ (np.percentile(image[:,:,2],95) - np.percentile(image[:,:,2],5)) #normalise and feed in the image
                batch_labels[folder, int(t[folder + ((num_batches-1)*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [68]:
curr_dt_time = datetime.datetime.now()
train_path = '/kaggle/input/gesture-recognition-upgrad/Project_data/train'
val_path = '/kaggle/input/gesture-recognition-upgrad/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [69]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Dropout, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


model = Sequential()
model.add(Conv3D(32, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,120,120,3)))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(1,1,1)))

model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Dropout(0.25))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Dropout(0.25))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('elu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(256, activation='elu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
#write your model here

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [70]:
optimiser = keras.optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

print (model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_22 (Conv3D)           (None, 18, 120, 120, 32)  2624      
_________________________________________________________________
batch_normalization_18 (Batc (None, 18, 120, 120, 32)  128       
_________________________________________________________________
activation_22 (Activation)   (None, 18, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_22 (MaxPooling (None, 17, 119, 119, 32)  0         
_________________________________________________________________
conv3d_23 (Conv3D)           (None, 17, 119, 119, 64)  55360     
_________________________________________________________________
batch_normalization_19 (Batc (None, 17, 119, 119, 64)  256       
_________________________________________________________________
activation_23 (Activation)   (None, 17, 119, 119, 64) 

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [71]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [72]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR =  ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.0001)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [73]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [74]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /kaggle/input/gesture-recognition-upgrad/Project_data/train ; batch size = 20
batch 0
Epoch 1/30
batch 1
32/34 [===========================>..] - ETA: 4s - loss: 4.9603 - categorical_accuracy: 0.2438last batch 3
num_batches 33
34/34 [==============================] - ETA: 0s - loss: 4.8653 - categorical_accuracy: 0.2428Source path =  /kaggle/input/gesture-recognition-upgrad/Project_data/val ; batch size = 20
batch 0
batch 1
batch 2
batch 3
batch 4
batch 0
34/34 [==============================] - 99s 3s/step - loss: 4.8653 - categorical_accuracy: 0.2428 - val_loss: 1.7906 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2022-12-0711_29_12.945522/model-00001-4.86527-0.24284-1.79061-0.20000.h5
Epoch 2/30
batch 1
32/34 [===========================>..] - ETA: 4s - loss: 1.9425 - categorical_accuracy: 0.3266last batch 3
num_batches 33
34/34 [==============================] - ETA: 0s - loss: 1.9493 - categorical_accuracy: 0.3228batch 1
batch 2
batch 3